In [1]:
import numpy as np
import sys, os
cwd_split = os.getcwd().split(os.sep)
root = os.path.abspath("")
sys.path.insert(0, f"{root}/build/python")

import mitsuba as mi
# mi.set_variant("cuda_ad_rgb")
mi.set_variant('cuda_ad_rgb_double', 'llvm_ad_rgb_double')
import drjit as dr

from matplotlib import pyplot as plt

from gaussnewton_test import conjugate_gradient, AlbedoScene

In [2]:
# def render_texture(texture, spp=256, seed=1):
#     params[key] = texture
#     params.update()
#     return mi.render(scene, params, spp=spp, seed=seed, seed_grad=seed+1)

In [3]:
# from mitsuba.scalar_rgb import Transform4f as T

filenames = [
    './tutorials/scenes/textures/image_001.png',
    './tutorials/scenes/textures/image_002.png',
    './tutorials/scenes/textures/image_003.png',
    './tutorials/scenes/textures/image_004.png'
]

# textures = [
#     mi.TensorXf(mi.Bitmap(f).convert(mi.Bitmap.PixelFormat.RGB, mi.Struct.Type.Float32))
#     for f in filenames
# ]

res = 128
print(res)
render_res = 32
print(render_res)
scene_init = mi.load_dict({
    'type': 'scene',
    'integrator': {'type': 'prb'},
    'sensor': {
        'type': 'perspective',
        'fov': 60,
        'to_world': mi.ScalarTransform4f.look_at(
            origin=[0, 0, -2],
            target=[0, 0, 0],
            up=[0, -1, 0]
        ),
        'film': {
            'type': 'hdrfilm',
            'width': render_res,
            'height': render_res,
            #'filter': { 'type': 'box' }
        }
    },
    'textured_plane': {
        'type': 'rectangle',
        'to_world': mi.ScalarTransform4f.scale(1.2),
        'bsdf': {
            'type': 'twosided',
            'nested': {
                'type': 'diffuse',
                'reflectance': {
                    'type': 'bitmap',
                    'filename': filenames[0]
                },
            }
        }
    },
    # 'glass_sphere': {
    #     'type': 'sphere',
    #     'to_world': mi.ScalarTransform4f.translate([0, 0, -1]).scale(0.45),
    #     'bsdf': {
    #         'type': 'dielectric',
    #         'int_ior': 1.06,
    #     },
    # },
    'light': {
        'type': 'constant',
    }
})

128
32


In [4]:
ref_images = mi.render(scene_init)
mi.util.convert_to_bitmap(ref_images)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [32, 32],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 3 KiB of image data ]
]

In [5]:

params = mi.traverse(scene_init)
key = 'textured_plane.bsdf.brdf_0.reflectance.data'
print(params[key])
# Save the original value
param_ref = mi.TensorXf(params[key])

# Set another color value and update the scene
params[key] = dr.full(mi.TensorXf, 0.01, shape=(res, res, 3))
params.update();
#print(params[key])


# params['sensor.film.size'] = mi.ScalarVector2u([64, 64])
# image_init = mi.render(scene, spp=128)
# print(image_init)
# mi.util.convert_to_bitmap(image_init)

scene_edit = {
    'type': 'scene',
    'integrator': {'type': 'prb'},
    'sensor': {
        'type': 'perspective',
        'fov': 60,
        'to_world': mi.ScalarTransform4f.look_at(
            origin=[0, 0, -2],
            target=[0, 0, 0],
            up=[0, -1, 0]
        ),
        'film': {
            'type': 'hdrfilm',
            'width': render_res,
            'height': render_res,
            #'filter': { 'type': 'box' }
        }
    },
    'textured_plane': {
        'type': 'rectangle',
        'to_world': mi.ScalarTransform4f.scale(1.2),
        'bsdf': {
            'type': 'twosided',
            'nested': {
                'type': 'diffuse',
                'reflectance': {
                    'type': 'bitmap',
                    'filename': filenames[0]
                },
            }
        }
    },
    'glass_sphere': {
        'type': 'sphere',
        'to_world': mi.ScalarTransform4f.translate([0, 0, -1]).scale(0.45),
        'bsdf': {
            'type': 'dielectric',
            'int_ior': 1.06,
        },
    },
    'light': {
        'type': 'constant',
    }
}

image_init = mi.render(scene_init, spp=128)
mi.util.convert_to_bitmap(image_init)

TensorXf64(shape=(128, 128, 3))


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [32, 32],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 3 KiB of image data ]
]

In [6]:
scene = mi.load_dict(scene_edit)
params_edit = mi.traverse(scene)
# key = {'shape1.emitter.radiance.value'}

# Instantiate the scene manager
keys = {'textured_plane.bsdf.brdf_0.reflectance.data'}
experiment_idx = -2
sc = AlbedoScene(scene, params_edit, keys, ref_images = ref_images, _EXP_IDX = experiment_idx)

# %%
# RED = '\033[91m'
# RESET = '\033[0m'
iteration_count = 20
image, loss = sc.eval_func()
for it in range(iteration_count):

    # Compute the Gauss-Newton descent direction
    descent_dir = conjugate_gradient(sc.values(), loss, sc)

    # Optimizer: take a Gauss Newton descent step
    sc.step(descent_dir)

    # Compute the loss using the current parameter values
    image, loss = sc.eval_func()

images = mi.render(scene)
mi.util.convert_to_bitmap(images)

49152
alpha = [0.04802791403391033]
   | r_curr = [0.0, 0.0, 0.0, 0.0, 0.0, .. 49142 skipped .., -0.00027500205596346693, -0.0003457576438058329, 0.0, 0.0, 0.0]
   | r_next = [0.0, 0.0, 0.0, 0.0, 0.0, .. 49142 skipped .., -3.0808845771796674e-19, 2.1367514399270956e-20, 0.0, 0.0, 0.0]
   | dir_x  = [-0.0, -0.0, -0.0, -0.0, -0.0, .. 49142 skipped .., 1.3207775102961987e-05, 1.660601839327393e-05, -0.0, -0.0, -0.0]
   | d_curr = [-0.0, -0.0, -0.0, -0.0, -0.0, .. 49142 skipped .., 0.00027500205596346693, 0.0003457576438058329, -0.0, -0.0, -0.0]
Conjugate Gradient converged after 0 iterations
Current residual is:  5.675042525938066e-13
alpha = [0.35896457284181926]
   | r_curr = [0.0, 0.0, 0.0, 0.0, 0.0, .. 49142 skipped .., -6.488151228722813e-05, -7.817029375950546e-05, 0.0, 0.0, 0.0]
   | r_next = [0.0, 0.0, 0.0, 0.0, 0.0, .. 49142 skipped .., -2.690402402982663e-20, 1.514730457786028e-19, 0.0, 0.0, 0.0]
   | dir_x  = [-0.0, -0.0, -0.0, -0.0, -0.0, .. 49142 skipped .., 2.329016434351609

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [32, 32],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 3 KiB of image data ]
]